# Purpose

The goal of this project is to create a dashboard to display data from a dataset. First we need to clean our data, then we can display it. The dataset used can be found [here](https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists). I'll be including the metadata for ease of use. Please note that I will only be using the training dataset, since my ultimate goal is not to predict data but to display it in a meaningful way. Also, I will be doing my best to maintain the accuracy of the data, but since there are many NaN rows, this may not be possible. It's not a big issue however, since my goal is only to display data, not analysis.

# Data Description

A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company. Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates. Information related to demographics, education, experience are in hands from candidates signup and enrollment.

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

The whole data divided to train and test . Target isn't included in test but the test target values data file is in hands for related tasks. A sample submission correspond to enrollee_id of test set provided too with columns : enrollee _id , target

Note:

The dataset is imbalanced.
Most features are categorical (Nominal, Ordinal, Binary), some with high cardinality.
Missing imputation can be a part of your pipeline as well.
Features

* enrollee_id : Unique ID for candidate
* city: City code
* city_ development _index : Developement index of the city (scaled)
* gender: Gender of candidate
* relevent_experience: Relevant experience of candidate
* enrolled_university: Type of University course enrolled if any
* education_level: Education level of candidate
* major_discipline :Education major discipline of candidate
* experience: Candidate total experience in years
* company_size: No of employees in current employer's company
* company_type : Type of current employer
* lastnewjob: Difference in years between previous job and current job
* training_hours: training hours completed
* target: 0 – Not looking for job change, 1 – Looking for a job change

# Data Cleaning

In [1]:
import pandas as pd
import numpy as np

In [2]:
jobdata = pd.read_csv('aug_train.csv')
jobdata

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [3]:
# Checking nans for all columns
jobdata.isna().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

Looks enrolee_id, city, city_development_index, relevent_experience, training_hours, and target have data values that are complete. First thing we need to do is deal with the columns that have nans, or we will have issues.

In [4]:
# Exploring the gender column
jobdata['gender'].value_counts(dropna=False)

Male      13221
NaN        4508
Female     1238
Other       191
Name: gender, dtype: int64

We will be changing the NaN gender values to No Reponse Given, since there are is a large amount of data with NaN values. It would eliminate too much data if we cut all those values out entirely, and since the goal of this project is simply to create a dashboard, the signifigance of having incorrect data will not affect our goal.

In [5]:
jobdata['gender'] = jobdata['gender'].fillna('No Response Given')

In [6]:
# Exploring the enrolled_university column
jobdata['enrolled_university'].value_counts(dropna=False)

no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
Name: enrolled_university, dtype: int64

It looks like most individuals are not currently enrolled in university courses. For that reason we will replace the NaN values with no_enrollement. Enrolling is also an optional choice as well, unlike gender which each individual must have, therefore it makes sense to replace NaN values with the most common value.

In [7]:
jobdata['enrolled_university'] = jobdata['enrolled_university'].fillna('no_enrollment')

In [8]:
# Exploring the education_level column
jobdata['education_level'].value_counts(dropna=False)

Graduate          11598
Masters            4361
High School        2017
NaN                 460
Phd                 414
Primary School      308
Name: education_level, dtype: int64

I had a feeling that having no education (an NaN value) would also mean that you would have no major_discipline as well.

In [9]:
test1 = jobdata['education_level'].fillna(1)

In [10]:
test2 = jobdata['major_discipline'].fillna(1)

In [11]:
test1.eq(test2).value_counts()

False    18698
True       460
dtype: int64

After some testing it looks like that assumption was correct. We can then replace the NaN values in education_level with "No Education" as they exactly correspond to the major_discipline NaN values. This makes sense since if you have no education, you cannot have a major_discipline. We also change the corresponding major_discipline as well.

In [12]:
jobdata['education_level'] = jobdata['education_level'].fillna('No Education')

In [13]:
# find where education_level == 'No education', and change the corresponding rows in major_discipline to 'No Major'
jobdata.loc[jobdata['education_level'] == 'No Education', 'major_discipline'] = 'No Major'

In [14]:
# Exploring major_discipline
jobdata['major_discipline'].value_counts(dropna=False)

STEM               14492
NaN                 2353
No Major             683
Humanities           669
Other                381
Business Degree      327
Arts                 253
Name: major_discipline, dtype: int64

Using the same logic we change the major_discipline of 'High School' education_level and 'Primary School' to 'No Major' as well, since individuals with these education levels would have NaNs for major_discipline,.

In [15]:
# find where education_level == 'High School', and change the corresponding rows in major_discipline to 'No Major'
jobdata.loc[jobdata['education_level'] == 'High School', 'major_discipline'] = 'No Major'
# find where education_level == 'Primary School', and change the corresponding rows in major_discipline to 'No Major'
jobdata.loc[jobdata['education_level'] == 'Primary School', 'major_discipline'] = 'No Major'

In [16]:
jobdata['major_discipline'].value_counts(dropna=False)

STEM               14492
No Major            3008
Humanities           669
Other                381
Business Degree      327
Arts                 253
NaN                   28
Name: major_discipline, dtype: int64

We have now chopped down our NaN values to just a mere 28. Now lets take a look at the remaining rows where major_discipine is NaN

In [17]:
jobdata.loc[jobdata['major_discipline'].isna()]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
391,12038,city_90,0.698,Male,Has relevent experience,Full time course,Masters,NaN,NaN,NaN,NaN,NaN,44,1.0
1771,1485,city_103,0.920,No Response Given,Has relevent experience,no_enrollment,Graduate,NaN,15,50-99,Pvt Ltd,>4,42,0.0
3796,2946,city_21,0.624,No Response Given,No relevent experience,Full time course,Graduate,NaN,2,NaN,Pvt Ltd,1,50,1.0
3923,22935,city_136,0.897,No Response Given,No relevent experience,no_enrollment,Graduate,NaN,3,NaN,NaN,NaN,92,0.0
4859,23075,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,NaN,>20,1000-4999,Public Sector,>4,7,0.0
5190,16615,city_162,0.767,No Response Given,Has relevent experience,no_enrollment,Graduate,NaN,16,NaN,NaN,NaN,43,0.0
6405,2874,city_41,0.827,No Response Given,Has relevent experience,Part time course,Masters,NaN,18,10000+,Pvt Ltd,>4,49,1.0
7816,28855,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,NaN,10,10000+,Pvt Ltd,3,14,0.0
8272,18836,city_136,0.897,No Response Given,No relevent experience,Full time course,Graduate,NaN,3,NaN,NaN,never,37,0.0
9038,17738,city_99,0.915,No Response Given,No relevent experience,Full time course,Graduate,NaN,5,NaN,NaN,1,18,1.0


We change the rest to "other" just for ease of use.

In [18]:
jobdata['major_discipline'] = jobdata['major_discipline'].fillna('Other')

We have now cleaned up the 'education_level' column of our dataset.

In [19]:
# Exploring the experience column
jobdata['experience'].value_counts(dropna=False)

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
NaN      65
Name: experience, dtype: int64

Looks like there are only 65 NaN values. Lets take a look at them.

In [20]:
jobdata.loc[jobdata['experience'].isna()]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
135,23947,city_103,0.920,No Response Given,No relevent experience,no_enrollment,Phd,STEM,NaN,NaN,NaN,NaN,70,0.0
391,12038,city_90,0.698,Male,Has relevent experience,Full time course,Masters,Other,NaN,NaN,NaN,NaN,44,1.0
1589,17642,city_134,0.698,No Response Given,No relevent experience,no_enrollment,No Education,No Major,NaN,NaN,NaN,never,33,0.0
2727,14965,city_103,0.920,No Response Given,Has relevent experience,no_enrollment,Masters,STEM,NaN,NaN,NaN,NaN,70,0.0
2859,24875,city_103,0.920,No Response Given,No relevent experience,Full time course,Graduate,STEM,NaN,10/49,NaN,1,41,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18039,9158,city_21,0.624,No Response Given,Has relevent experience,no_enrollment,Masters,STEM,NaN,5000-9999,NaN,1,13,1.0
18055,32626,city_150,0.698,Other,Has relevent experience,no_enrollment,No Education,No Major,NaN,10/49,NaN,1,51,1.0
18143,2299,city_21,0.624,No Response Given,Has relevent experience,no_enrollment,Graduate,STEM,NaN,10/49,NaN,NaN,182,1.0
18790,21862,city_114,0.926,No Response Given,Has relevent experience,no_enrollment,Graduate,STEM,NaN,50-99,NGO,1,138,0.0


Logically, if an individual has NaN for company_size and company_type, than that would also mean they would have NaN for experience. We can use this inference to help fill out some NaN values in the experience column.

In [21]:
# find where company_size == NaN & company_type == NaN, and change the corresponding rows in experience to 0
jobdata.loc[pd.isna(jobdata['company_size']) & pd.isna(jobdata['company_type']), 'experience'] = 0

In [22]:
jobdata.loc[jobdata['experience'].isna()].head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
2859,24875,city_103,0.920,No Response Given,No relevent experience,Full time course,Graduate,STEM,NaN,10/49,NaN,1,41,1.0
3086,19706,city_11,0.550,No Response Given,Has relevent experience,Full time course,Primary School,No Major,NaN,<10,Early Stage Startup,NaN,62,1.0
3760,21042,city_103,0.920,No Response Given,Has relevent experience,Full time course,Graduate,STEM,NaN,50-99,Funded Startup,>4,24,0.0
4003,16736,city_103,0.920,No Response Given,No relevent experience,no_enrollment,Masters,STEM,NaN,10000+,NaN,NaN,103,0.0
4207,9306,city_67,0.855,Male,Has relevent experience,no_enrollment,Masters,Business Degree,NaN,NaN,Pvt Ltd,>4,96,1.0


In [23]:
jobdata['experience'].value_counts(dropna=False)

0      5360
>20    2422
5      1007
6       908
4       902
3       816
10      799
9       768
7       757
2       630
8       623
15      571
11      527
14      488
16      413
12      403
13      327
1       299
17      271
<1      253
19      242
18      219
20      108
NaN      45
Name: experience, dtype: int64

Now it doesn't seem possible to extrapolate any further information. We drop the remaining rows with experience == NaN 

In [24]:
jobdata = jobdata[jobdata['experience'].notna()]

In [25]:
# Exploring company_size, company_type
jobdata['company_size'].value_counts(dropna=False)

NaN          5935
50-99        3078
100-500      2563
10000+       2011
10/49        1466
1000-4999    1323
<10          1304
500-999       874
5000-9999     559
Name: company_size, dtype: int64

In [26]:
jobdata['company_type'].value_counts(dropna=False)

Pvt Ltd                9796
NaN                    6124
Funded Startup          999
Public Sector           953
Early Stage Startup     601
NGO                     519
Other                   121
Name: company_type, dtype: int64

At this point it can be observed that if an individual has never had a job, then they will neither have any company_size or company_type. Therefore, it can be assumed that if an individual has NaN values for company_size and company_type, they have never held a job before. We will then change the NaN values to 0, and None respectively

In [27]:
# find where company_size == NaN & company_type == NaN, and change the corresponding rows in company_size and company_type to 0 and None respectively
# Made a copy of jobdata, to aviod SettingWithCopy warning
jobdf = jobdata.copy()
jobdf.loc[pd.isna(jobdf['company_size']) & pd.isna(jobdf['company_type']),['company_size','company_type']] = '0','None'

In [28]:
jobdf

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,0,0,None,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,No Response Given,No relevent experience,Full time course,Graduate,STEM,0,0,None,never,83,0.0
3,33241,city_115,0.789,No Response Given,No relevent experience,no_enrollment,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,0,0,None,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,0,0,None,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,No Major,<1,500-999,Pvt Ltd,2,97,0.0


In [29]:
jobdf['company_type'].value_counts(dropna=False)

Pvt Ltd                9796
None                   5360
Funded Startup          999
Public Sector           953
NaN                     764
Early Stage Startup     601
NGO                     519
Other                   121
Name: company_type, dtype: int64

In [30]:
jobdf['company_size'].value_counts(dropna=False)

0            5360
50-99        3078
100-500      2563
10000+       2011
10/49        1466
1000-4999    1323
<10          1304
500-999       874
NaN           575
5000-9999     559
Name: company_size, dtype: int64

The NaNs have been cut down quite a bit. From here, we can drop the rest of the NaN row since it is not possible to extrapolate data further.

In [31]:
jobdf = jobdf[jobdf['company_type'].notna()]

In [32]:
jobdf = jobdf[jobdf['company_size'].notna()]

In [33]:
# Checking nans for all columns
jobdf.isna().sum()

enrollee_id                 0
city                        0
city_development_index      0
gender                      0
relevent_experience         0
enrolled_university         0
education_level             0
major_discipline            0
experience                  0
company_size                0
company_type                0
last_new_job              354
training_hours              0
target                      0
dtype: int64

In [34]:
# Exploring last_new_job
jobdf['last_new_job'].value_counts(dropna=False)

1        7433
>4       3185
2        2742
never    2110
4         985
3         965
NaN       354
Name: last_new_job, dtype: int64

In [35]:
# Veiwing rows with last_new_job NaN
jobdf.loc[jobdf['last_new_job'].isna()]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
58,13478,city_21,0.624,No Response Given,Has relevent experience,Full time course,Graduate,STEM,2,10/49,Funded Startup,NaN,32,1.0
64,9572,city_11,0.550,No Response Given,No relevent experience,Full time course,High School,No Major,0,0,None,NaN,98,0.0
90,19128,city_61,0.913,Male,No relevent experience,Full time course,Graduate,STEM,0,0,None,NaN,140,0.0
135,23947,city_103,0.920,No Response Given,No relevent experience,no_enrollment,Phd,STEM,0,0,None,NaN,70,0.0
205,4478,city_11,0.550,No Response Given,Has relevent experience,no_enrollment,Graduate,STEM,5,50-99,Pvt Ltd,NaN,7,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18944,2678,city_104,0.924,No Response Given,No relevent experience,no_enrollment,Phd,STEM,>20,50-99,Pvt Ltd,NaN,27,0.0
19043,17966,city_114,0.926,No Response Given,No relevent experience,Full time course,High School,No Major,0,0,None,NaN,20,0.0
19106,18131,city_100,0.887,Male,No relevent experience,Full time course,Masters,STEM,6,100-500,Public Sector,NaN,8,0.0
19113,11622,city_19,0.682,No Response Given,No relevent experience,Full time course,Graduate,STEM,0,0,None,NaN,53,0.0


Similar to earlier, if an idividual did not have a job ever, then they would also not have a last_new_job. We will replace those last_new_job rows with the value "never".

In [36]:
# find where experience == '0', and change the corresponding rows in last_new_job to 'never'
jobdf.loc[jobdata['experience'] == 0, 'last_new_job'] = 'never'

In [37]:
jobdf.loc[jobdf['last_new_job'].isna()]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
58,13478,city_21,0.624,No Response Given,Has relevent experience,Full time course,Graduate,STEM,2,10/49,Funded Startup,NaN,32,1.0
205,4478,city_11,0.550,No Response Given,Has relevent experience,no_enrollment,Graduate,STEM,5,50-99,Pvt Ltd,NaN,7,1.0
575,24981,city_45,0.890,No Response Given,Has relevent experience,no_enrollment,Graduate,STEM,9,50-99,Pvt Ltd,NaN,62,0.0
902,1926,city_123,0.738,No Response Given,No relevent experience,no_enrollment,Masters,STEM,15,50-99,Pvt Ltd,NaN,44,0.0
909,12104,city_21,0.624,No Response Given,Has relevent experience,no_enrollment,Graduate,STEM,9,5000-9999,Pvt Ltd,NaN,14,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18648,27777,city_21,0.624,Male,Has relevent experience,no_enrollment,Graduate,STEM,4,10000+,Pvt Ltd,NaN,82,1.0
18772,7756,city_21,0.624,No Response Given,Has relevent experience,no_enrollment,Graduate,STEM,2,<10,Early Stage Startup,NaN,4,0.0
18863,10381,city_103,0.920,No Response Given,Has relevent experience,no_enrollment,Graduate,Humanities,19,10000+,Pvt Ltd,NaN,13,1.0
18944,2678,city_104,0.924,No Response Given,No relevent experience,no_enrollment,Phd,STEM,>20,50-99,Pvt Ltd,NaN,27,0.0


The remaining rows appear to be individuals who have never left thier current position and is in thier first job at the moment. They can also be classified as never having a last new job as well.

In [38]:
jobdf['last_new_job'] = jobdf['last_new_job'].fillna('never')

# Conclusion

With that, let's take a look at our cleaned data.

In [39]:
# Checking nans for all columns
jobdf.isna().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

There are no longer any NaN values in any of the columns. Please note that this was a very basic data cleaning opertaion. I would **NOT** clean data like this if my intent was to perform and analysis on the dataset. If I wanted to do an analysis, I would integrate NaN values that I could not logically determine into my analysis pipline instead. However, since the purpose of this project is to create a functional dashboard to display data, it doesn't matter too much.

In [44]:
# Export dataframe to csv
jobdf.to_csv('cleaneddata.csv')